####  Этап 1: Оценка проблемы — Сводный анализ отключений

#### На этом этапе мы анализируем таблицу `blackouts`, чтобы понять **масштаб, длительность и характер** проблемы отключения коммунальных услуг в целом.
 
#### **Цель:**
##### 1.  Оценить общее количество и продолжительность отключений.
##### 2.  Выявить самые проблемные типы услуг (электричество, вода, отопление).
##### 3.  Определить основные организации-инициаторы отключений.

In [ ]:
# 1.1. Настройка окружения и загрузка данных

# Используем библиотеку `sqlite3` для подключения к базе данных и `pandas` для анализа и обработки данных.

import sqlite3
import pandas as pd
import numpy as np

# Установка опций отображения
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Подключение к базе данных
DB_PATH = '../databases/dataset.db' 
conn = sqlite3.connect(DB_PATH)

# Загрузка таблицы blackouts
query_blackouts = "SELECT * FROM blackouts;"
blackouts_df = pd.read_sql_query(query_blackouts, conn)

print(f"Загружено записей в blackouts: {len(blackouts_df)}")
blackouts_df.head()

Загружено записей в blackouts: 25264


,id,start_date,end_date,description,type,initiator_name,source
0,f88cefa506f44ebf8f010b8681b5449e,2018-01-01 00:08:00,2018-01-01 09:00:00,"Авария на сети электроснабжения, ведутся восст...",electricity,МУПВ ВПЭС (электрические сети Ленинского района),Единая дежурная диспетчерская служба города (Л...
1,38ddf6852801fa90cc70f9770239961e,2018-01-01 00:24:00,2018-01-01 10:00:00,"Авария на электролинии, остановка работы насос...",cold_water,МУПВ ВПЭС (электрические сети Ленинского района),Единая дежурная диспетчерская служба города (Л...
2,53c570099fe380dce9e56e2ace9cfa9c,2018-01-01 10:44:00,2018-01-02 18:00:00,Авария в системе водоснабжения дома. Жителям н...,hot_water,"ООО ""Управляющая компания ""Регион-ЖКХ""","Аварийная служба ООО ""Мадикс"""
3,9c1b9ebbd9a698eef046b27cb3568745,2018-01-01 11:32:00,2018-01-01 15:00:00,"Авария на сети электроснабжения, ведутся восст...",electricity,МУПВ ВПЭС (электрические сети Фрунзенского рай...,Единая дежурная диспетчерская служба города (Ф...
4,8aa631cb343aac0731bbde806dfa6d8c,2018-01-01 11:33:00,2018-01-01 15:00:00,"Авария на электролинии, остановка работы насос...",hot_water,МУПВ ВПЭС (электрические сети Фрунзенского рай...,Единая дежурная диспетчерская служба города (Ф...


In [ ]:
# 1.2. Подготовка данных: Расчет длительности

# Конвертируем столбцы с датами в формат datetime и рассчитываем продолжительность каждого отключения в часах.

# Преобразование столбцов с датами в формат datetime
blackouts_df['start_date'] = pd.to_datetime(blackouts_df['start_date'])
blackouts_df['end_date'] = pd.to_datetime(blackouts_df['end_date'])

# Расчет длительности отключения (timedelta)
blackouts_df['duration'] = blackouts_df['end_date'] - blackouts_df['start_date']

# Расчет длительности в часах (для удобства анализа)
blackouts_df['duration_hours'] = blackouts_df['duration'].dt.total_seconds() / 3600

# Удаление записей с отрицательной или нулевой длительностью (ошибочные данные или мгновенные события)
blackouts_df = blackouts_df[blackouts_df['duration_hours'] > 0]

print(f"Количество записей после очистки: {len(blackouts_df)}")
blackouts_df[['start_date', 'end_date', 'duration', 'duration_hours']].describe()

Количество записей после очистки: 25264


,start_date,end_date,duration,duration_hours
count,25264,25264,25264,25264.000000
mean,2018-12-29 18:52:18.816497408,2018-12-31 02:06:20.649936640,1 days 07:14:01.833438885,31.233843
min,2018-01-01 00:08:00,2018-01-01 09:00:00,0 days 00:01:00,0.016667
25%,2018-07-08 09:36:00,2018-07-12 12:00:00,0 days 02:52:00,2.866667
50%,2018-12-27 16:36:00,2018-12-28 08:14:30,0 days 04:41:00,4.683333
75%,2019-06-18 11:14:00,2019-06-19 16:00:00,0 days 10:45:00,10.750000
max,2019-12-31 20:29:00,2020-01-09 12:00:00,355 days 23:13:00,8543.216667
std,NaN,NaN,6 days 16:21:31.850660568,160.358847


In [3]:
#  1.3. Сводный анализ длительности и частоты

# Рассчитаем ключевые метрики, чтобы оценить общий масштаб проблемы для пользователей.


# Общее количество отключений
total_blackouts = len(blackouts_df)

# Общая длительность отключений (в часах и днях)
total_duration_hours = blackouts_df['duration_hours'].sum()
total_duration_days = total_duration_hours / 24

# Статистика по длительности одного отключения
avg_duration = blackouts_df['duration_hours'].mean()
median_duration = blackouts_df['duration_hours'].median()
max_duration = blackouts_df['duration_hours'].max()

print("--- Общие метрики отключений ---")
print(f"Всего отключений: {total_blackouts:,}")
print(f"Общее время, проведенное без услуг (суммарно по всем отключениям):")
print(f"  {total_duration_hours:,.2f} часов, или {total_duration_days:,.2f} дней")
print("-" * 35)
print("Статистика длительности одного отключения (часы):")
print(f"  Средняя длительность: {avg_duration:.2f} ч.")
print(f"  Медианная длительность: {median_duration:.2f} ч. (более устойчивый показатель)")
print(f"  Максимальная длительность: {max_duration:.2f} ч.")

--- Общие метрики отключений ---
Всего отключений: 25,264
Общее время, проведенное без услуг (суммарно по всем отключениям):
  789,091.80 часов, или 32,878.83 дней
-----------------------------------
Статистика длительности одного отключения (часы):
  Средняя длительность: 31.23 ч.
  Медианная длительность: 4.68 ч. (более устойчивый показатель)
  Максимальная длительность: 8543.22 ч.


 **Вывод по сводному анализу:**

 Проблема отключений характеризуется высокой частотой (более 25 тысяч событий) и ощутимой продолжительностью (медиана 4.68 часа). Такое сочетание делает простое информирование о факте отключения недостаточным.

Ценность фичи предсказания заключается в следующем: зная, что типичное отключение длится почти 5 часов, пользователи, получившие прогноз, смогут заранее принять меры (например, перенести бытовые дела, изменить свое расписание дня, или уехать на время отключения). Это превращает неопределенность и вынужденный простой в управляемое ожидание, что напрямую повышает удовлетворенность пользователей.


In [4]:

# ## 1.4. Анализ по типу услуг (`type`)
# Распределение отключений по типам услуг (`electricity`, `water`, `heating`) для выявления наиболее критичных областей.

type_analysis = blackouts_df.groupby('type').agg(
    count=('id', 'count'),
    total_duration_hours=('duration_hours', 'sum'),
    average_duration_hours=('duration_hours', 'mean'),
    median_duration_hours=('duration_hours', 'median')
).sort_values(by='count', ascending=False)

# Добавление столбца с долей от общего числа отключений
type_analysis['count_share'] = (type_analysis['count'] / total_blackouts) * 100

print("--- Анализ отключений по типу услуги ---")
print(type_analysis.round(2))

--- Анализ отключений по типу услуги ---
             count  total_duration_hours  average_duration_hours  median_duration_hours  count_share
type                                                                                                
hot_water     9339             650733.10                   69.68                   6.00        36.97
cold_water    9109              93630.82                   10.28                   4.65        36.06
electricity   4745              21933.92                    4.62                   3.20        18.78
heat          2071              22793.97                   11.01                   5.25         8.20



Вывод по Анализу Отключений по Типу Услуг

Ключевые наблюдения и их значение:
1) Горячая вода (hot_water)	Доля в общем числе: 36.97%,	Медианная длительность: 6.00 ч.
Самый частый тип отключений и самый продолжительный по медиане. Предсказание критически важно, так как затрагивает наибольшее число событий и приносит наибольший дискомфорт (долгие отключения).

2) Холодная вода (cold_water)	Доля в общем числе: 36.06%, Медианная длительность: 4.65 ч.
Почти столь же часто, как горячая вода. Продолжительность также высока. Ключевой приоритет для предсказания, поскольку отсутствие холодной воды является серьезной бытовой проблемой.

3) Электричество (electricity)	Доля в общем числе: 18.78%, Медианная длительность: 3.20 ч.
Менее часто, но всё еще значительно. Имеет самую низкую медианную длительность. Тем не менее, критично для работы и жизнеобеспечения, поэтому должно быть включено в предсказание.

4) Отопление (heat)	Доля в общем числе: 8.20%, Медианная длительность: 5.25 ч.
Наименее частый, но имеет вторую по продолжительности медиану. Отключение отопления в холодное время года может быть самым критическим с точки зрения последствий.

Обоснование приоритетов фичи:

1) Приоритет №1: Водоснабжение (Горячая и Холодная вода): Составляют более 73% всех отключений. Это является основным фокусом для фичи предсказания, так как пользователи чаще всего сталкиваются именно с этой проблемой, и она длится долго (4.65–6.00 часов).

2) Приоритет №2: Отопление: Несмотря на низкую частоту, высокая медианная длительность (5.25 ч.) и критическая важность услуги, особенно в зимний период, делают предсказание времени отключения отопления высокоценным для пользователя.

3) Приоритет №3: Электричество: Имеет наименьшую медианную длительность, но высокая значимость электроэнергии для современных бытовых и рабочих нужд требует включения этого типа в фичу предсказания.

Таким образом, фича предсказания времени отключения должна быть универсальной для всех типов, но акцент на точности и оперативности следует делать в первую очередь на отключениях горячей и холодной воды.

In [5]:
# ## 1.5. Анализ по инициатору (`initiator_name`)
# Выявление организаций, которые чаще всего являются инициаторами работ и, как следствие, отключений.

initiator_analysis = blackouts_df.groupby('initiator_name').agg(
    count=('id', 'count'),
    total_duration_hours=('duration_hours', 'sum'),
    median_duration_hours=('duration_hours', 'median')
).sort_values(by='count', ascending=False).head(10)

print("--- ТОП-10 Инициаторов отключений ---")
print(initiator_analysis.round(2))

--- ТОП-10 Инициаторов отключений ---
                                                 count  total_duration_hours  median_duration_hours
initiator_name                                                                                     
МУПВ ВПЭС (электрические сети)                    2431              13134.85                   4.12
КГУП «Приморский водоканал»                       1415              13918.03                   7.13
ООО "Мингородок"                                   875               5716.98                   3.72
ООО "Невельского"                                  777               7795.53                   4.10
МУПВ ВПЭС (тепловые сети Первомайского района)     751              98234.93                  14.27
МУПВ ВПЭС (тепловые сети)                          699             122782.58                  24.02
ООО "УК "Комплекс коммунальных услуг"              624              11153.87                   6.57
ООО "Управляющая компания Первомайского района"    592        

Вывод по Анализу Инициаторов Отключений
Анализ ТОП-10 инициаторов отключений показывает, какие организации генерируют наибольшее количество проблем и определяют самые продолжительные простои, что является важным контекстом для обоснования фичи предсказания.

Ключевые наблюдения:

Лидеры по частоте:

Наибольшее количество отключений приходится на МУПВ ВПЭС (электрические сети) (2431 случай) и КГУП «Приморский водоканал» (1415 случаев). Это логично, поскольку они отвечают за наиболее частые типы отключений (электричество и вода), как показано в Выводе 1.4.

Лидеры по продолжительности (по медиане):

МУПВ ВПЭС (тепловые сети) (медиана 24.02 ч.) и МУПВ ВПЭС (тепловые сети Первомайского района) (медиана 14.27 ч.) ответственны за отключения с самой высокой медианной длительностью.

Эти данные указывают, что хотя отключения тепла и горячей воды (за которые они в основном отвечают) происходят реже, чем отключения электроэнергии, их устранение занимает значительно больше времени.

Управляющие компании (УК):

В списке также присутствует значительное количество управляющих компаний (например, ООО "Мингородок", ООО "Невельского", ООО "УК "Комплекс коммунальных услуг"), что говорит о том, что существенная доля проблем связана не только с магистральными сетями, но и с внутридомовыми авариями или работами, которые проводятся самими УК.

Значение для фичи предсказания:
Выявление инициаторов позволяет понять, что проблема имеет две основные природы:

1) Массовые и относительно короткие проблемы (Электричество/Вода от крупных сетевых организаций).

2) Редкие, но крайне продолжительные проблемы (Отопление/Горячая вода от тепловых сетей).

Вывод: Поскольку отключения, инициированные тепловыми сетями ВПЭС, длятся многократно дольше (медиана до 24 часов) по сравнению с другими (медиана 3-7 часов), предсказание времени восстановления для этих событий имеет наивысшую пользовательскую ценность. Необходимость провести сутки без отопления или горячей воды требует максимально точного и своевременного информирования.

### Сводный Итог Этапа 1
Анализ таблицы blackouts подтвердил, что фича предсказания времени отключения является значимым улучшением:

Масштаб: Более 25 тысяч отключений и почти 800 тысяч совокупных часов простоя демонстрируют высокий риск для пользователей.

Актуальность: Типичное отключение длится около 5 часов (медиана), что достаточно долго для создания серьезных неудобств.

Критичность: Проблемы с теплом и водой (особенно инициированные тепловыми сетями МУПВ ВПЭС) являются самыми продолжительными (медиана до 24 часов), что делает предсказание для них  важным для планирования жизни и минимизации ущерба